In [14]:
import pandas as pd
from machine_learning.readability_scorer import ReadabilityScorer
from machine_learning.sentiment_model import SentimentModel
from machine_learning.model import FakeNewsClassifier
import datasets

from config import base_columns

In [7]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
valid = pd.read_csv("data/valid.csv")

In [15]:
dataset = "chengxuphd/liar2"
dataset = datasets.load_dataset(dataset)
statement_train, y_train = dataset["train"]["statement"], dataset["train"]["label"]
statement_val, y_train = dataset["validation"]["statement"], dataset["validation"]["label"]
statement_test, y_test = dataset["test"]["statement"], dataset["test"]["label"]

Generating test split: 100%|██████████| 2296/2296 [00:00<00:00, 17805.07 examples/s]


In [16]:
statement_train

['90 percent of Americans "support universal background checks" for gun purchases.',
 'Last year was one of the deadliest years ever for law enforcement officers.',
 'Bernie Sanders\'s plan is "to raise your taxes to 90 percent.',
 'Voter ID is supported by an overwhelming majority of NYers, from all across the state, walks of life, & political parties.',
 'Says Barack Obama "robbed Medicare (of) $716 billion to pay for ... Obamacare.',
 'Video says COVID-19 vaccines are "weapons of mass destruction" that "could wipe out the human race.',
 'Says Jeff Reardon cut elementary school music classes, art instruction, teaching positions, basketball and volleyball programs, and he voted to raise school lunch prices.',
 'One in three women is sexually assaulted on the dangerous journey north.',
 "It's actually the safest time to fly.",
 'We know dangerous criminals can slip through immigration at airports just like ours. Yet Joe Donnelly stood right here with the radical left against all effort